# Make BoW, TFIDF, Lsi

In [1]:
from tfidf_pm import *
pubmed_corpus = PubmedCorpus()

In [ ]:
pubmed_corpus.load_corpus()

In [ ]:
pubmed_corpus.dictionary.doc2bow(['yo', 'yellow', 'circumference', 'obesity'])

In [ ]:
pubmed_tfidf = models.TfidfModel(pubmed_corpus, normalize=True)
pubmed_tfidf.save(os.path.join(SAVE_LOCATION, 'pubmed_tfidf'))

pubmed_corpus_tfidf = pubmed_tfidf[pubmed_corpus]
pubmed_corpus_tfidf.save(os.path.join(SAVE_LOCATION, 'pubmed_corpus_tfidf'))

pubmed_lsi = models.LsiModel(pubmed_corpus_tfidf, 
                             id2word=pubmed_corpus.dictionary, 
                             num_topics=200)
pubmed_corpus_lsi = pubmed_lsi[pubmed_corpus_tfidf]
pubmed_lsi.save(os.path.join(SAVE_LOCATION, 'pubmed_lsi'))

##### save dictionary

In [ ]:
pubmed_corpus.dictionary.save('./pmc_models_serialized/pubmed_corpus.dict')

##### saved market matrix format (iterates through corpus on creation, .... lazy)

In [ ]:
corpora.MmCorpus.serialize('./pmc_models_serialized/pubmed_corpus.mm', pubmed_corpus)

##### load dictionary

In [ ]:
from tfidf_pm import *

In [ ]:
pubmed_corpus = corpora.MmCorpus('./pmc_models_serialized/pubmed_corpus.mm')

In [ ]:
pubmed_dict = corpora.Dictionary.load('./pmc_models_serialized/pubmed_corpus.dict')

##### Load tfidf_model

In [ ]:
pubmed_tfidf = models.tfidfmodel.TfidfModel.load('./pmc_models_serialized/pubmed_tfidf')
# pubmed_corpus_tfidf = models.tfidfmodel.TfidfModel.load('./pmc_models_serialized/pubmed_corpus_tfidf')

##### Load LSI model and corpus

In [ ]:
pubmed_lsi = models.LsiModel.load('./pmc_models_serialized/pubmed_lsi')
pubmed_corpus_lsi = models.LsiModel.load('./pmc_models_serialized/pubmed_corpus_lsi')

### Folding in a new document and loading models

tfidf = models.tfidfmodel.TfidfModel.load('tfidf')

lsi = models.LsiModel.load('lsi')

Create an index transformer that calculates similarity based on our space
 
test_index1 = similarities.MatrixSimilarity(test_lsi1)

##### newdoc_loading

In [ ]:
new_doc = """
The abnormal position of the testis is the most common congenital anomaly of genitalia in men, with two different periods of presentation: either early in life, commonly called congenital undescended testes, or a few years later, called acquired or ascending testes. In the second group, it seems that the testis has been positioned properly at an early stage of growth, but has subsequently left its normal scrotal position to occupy the inguinal area. This group is quite controversial, as some authors considered these cases to be misdiagnoses owing to an error in physical examination.1 The cumulative experience of qualified examiners, however, suggests that this is a real phenomenon,2 which explains the increasing proportion of orchidopexies performed in late infancy.3"""
new_doc = utils.tokenize(new_doc)
new_doc_bow = pubmed_dict.doc2bow(new_doc)
new_doc_bow = [i for i in new_doc_bow if i not in STOP_WORDS]
new_doc_tfidf = pubmed_tfidf[new_doc_bow]

In [ ]:
new_doc_lsi = pubmed_lsi[new_doc_tfidf]

In [ ]:
new_doc_lsi

##### Similarity object and comparison

In [ ]:
# pubmed_corpus_lsi = pubmed_lsi[pubmed_corpus_tfidf]

In [ ]:
pubmed_lsi_in_memory = [i for i in pubmed_corpus_lsi]

In [ ]:
one_pubmed_lsi_doc = pubmed_lsi_in_memory[0]

In [ ]:
one_pubmed_lsi_sim = pubmed_sim[one_pubmed_lsi_doc]

In [ ]:
one_pubmed_lsi_sim

In [ ]:
pubmed_sim = similarities.Similarity.load('./pmc_models_serialized/pubmed_sim')

In [ ]:
new_doc_sims = pubmed_sim[new_doc_lsi]

In [ ]:
file_name_sims = zip(new_doc_sims.tolist(), pubmed_corpus_lsi.corpus.corpus.document_file_names)

In [ ]:
file_name_sims = sorted(file_name_sims, key=lambda x: x[0], reverse=True)

In [ ]:
file_name_sims[:5]

In [ ]:
??similarities.Similarity

In [ ]:
??interfaces.SimilarityABC

In [ ]:
??pubmed_corpus.dictionary.load

In [ ]:
??corpora.mmcorpus.MmCorpus

In [ ]:
??models.TfidfModel

In [ ]:
??corpora.TextCorpus

In [ ]:
??interfaces.CorpusABC

In [ ]:
??utils.SaveLoad

In [ ]:
dictionary = corpora.Dictionary()

In [ ]:
dictionary.doc2bow(['hello', 'something'])

In [ ]:
class thing:
    def __init__(self):
        self.array = [(1, 2, 3), (3,3, 3) , (2, 2, 2)]
    
    def __iter__(self):
        for i, j, k in self.array:
            yield i * 2
            
    def load_stuff(self):
        for i in self:
            print i
            
    def again(self):
        for i in self:
            print i

In [ ]:
a = thing()

In [ ]:
a.load_stuff()

In [ ]:
# class PubmedCorpus(corpora.TextCorpus):
    
#     def get_texts(self):
#         directory = os.walk(self.input)
#         for root, dirs, files in directory:
#             for file_name in files:
#                 file_path = os.path.join(root, file_name)
#                 with open(file_path, 'rb') as f:
#                     file_string = to_unicode_or_bust(f.read().lower())
#                     file_tokenized = word_tokenize(file_string)
#                     file_no_stops = [word for word in file_tokenized if word not in STOP_WORDS]
#                     yield file_no_stops

# class PubmedCorpus(corpora.TextCorpus):
        
#     def get_texts(self):
#         for file_path in self.input:
#             with open(file_path, 'rb') as f:
#                 file_string = to_unicode_or_bust(f.read().lower())
#                 file_tokenized = word_tokenize(file_string)
#                 file_no_stops = [word for word in file_tokenized if word not in STOP_WORDS]
#                 yield file_no_stops
                    
                
# def load_article_paths():
#     for root, dirs, files in os.walk('./pmc_data/pmc_text_files/'):
#         for name in files:
#             ARTICLE_FILE_PATHS.append(os.path.join(root, name))
#     genre_folders_left = len(GENRE_FOLDERS)
#     completed_genre_folders = 0 
#     for genre_folder in GENRE_FOLDERS:
#         completed_genre_folders += 1
#         genre_folder_path = os.path.join(DATA_PATH, genre_folder)
#         genre_file_list = os.listdir(genre_folder_path)
#         for article_file_title in genre_file_list:
#             article_file_path = os.path.join(genre_folder_path, article_file_title)
#             if os.path.isfile(article_file_path):
#                 ARTICLE_FILE_TITLES.append(article_file_title)
#                 ARTICLE_FILE_PATHS.append(article_file_path)
# #                 with open(article_file_path, 'rb') as f:
# #                     document = f.read()
# #                     ARTICLE_DOCUMENT_LIST.append(document)

#                 print "done with: ", article_file_title
#                 print "progress: ", completed_genre_folders / float(genre_folders_left)
#             else:
#                 sub_article_folder_list = os.listdir(article_file_path)
#                 for sub_article_file_title in sub_article_folder_list:
#                     sub_article_file_path = os.path.join(article_file_path, 
#                             sub_article_file_title)
#                     ARTICLE_FILE_TITLES.append(sub_article_file_title)
#                     ARTICLE_FILE_PATHS.append(sub_article_file_path)
# #                     with open(sub_article_file_path, 'rb') as f:
# #                         document = f.read()
# #                         ARTICLE_DOCUMENT_LIST.append(document)

### Pubmed Corpus from corpora.TextCorpus

In [ ]:
# class PubmedCorpus(corpora.TextCorpus):
#     def __init__(self, input=None, data_folder=DATA_PATH):
#         super(TextCorpus, self).__init__()
#         self.input = None
#         self.data_folder = os.walk(data_folder)
#         self.dictionary = Dictionary()
#         self.metadata = False        
#         if data_folder is not None:
#             self.dictionary.add_documents
    
#     def __iter__(self):
        
        
#     def get_texts(self, file_name):
#         with open(file_name, 'rb') as doc_file:
#             doc = doc_file.read()
#             doc_tokenized = utils.tokenize(doc, lowercase=True)
#             doc_tokenized = [word for word in doc_tokenized if word not in STOP_WORDS]
#             yield doc_tokenized

                    
# #         for doc_file in self.input:
# #             with open(doc_file, 'rb') as doc:
# #                 doc_text = doc.read()
# #                 tokenized_text = [token for token in utils.tokenize(doc_text, 
# #                                                                     lowercase=True)]
# #                 tokenized_text = [word for word in tokenized_text if word not in STOP_WORDS]
# #                 self.dictionary.add_documents([tokenized_text])
# #                 yield self.dictionary.doc2bow(tokenized_text, allow_update=False)
                
        
        
        
        
# #         directory = os.walk(self.input)
# #         for root, dirs, files in directory:
# #             for file_name in files:
# #                 file_path = os.path.join(root, file_name)
# #                 with open(file_path, 'rb') as f:
# #                     file_string = to_unicode_or_bust(f.read().lower())
# #                     file_tokenized = word_tokenize(file_string)
# #                     file_no_stops = [word for word in file_tokenized if word not in STOP_WORDS]
# #                     yield file_no_stops